In [ ]:
import pandas as pd
import json

In [3]:
df = pd.read_parquet("../data/train_part_0001.snappy.parquet", engine="pyarrow")

In [11]:
print(f"Размер: {df.shape}\n")  

print(df.info())

Размер: (500000, 27)

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 0 to 499999
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_item_id           500000 non-null  object 
 1   cand_item_id           500000 non-null  object 
 2   group_id               500000 non-null  int64  
 3   action_date            500000 non-null  object 
 4   base_title             500000 non-null  object 
 5   cand_title             500000 non-null  object 
 6   base_description       500000 non-null  object 
 7   cand_description       500000 non-null  object 
 8   base_category_name     499997 non-null  object 
 9   cand_category_name     500000 non-null  object 
 10  base_subcategory_name  499997 non-null  object 
 11  cand_subcategory_name  500000 non-null  object 
 12  base_param1            480372 non-null  object 
 13  cand_param1            480086 non-null  object 
 14  base_param2        

In [12]:
print(df["is_double"].value_counts(normalize=True))

is_double
0    0.938822
1    0.061178
Name: proportion, dtype: float64


In [ ]:
categorical_columns = [
    'base_category_name', 'cand_category_name',
    'base_subcategory_name', 'cand_subcategory_name',
    'base_param1', 'cand_param1',
    'base_param2', 'cand_param2'
]

for col in categorical_columns:
    print(f"\n--- {col} ---")
    print(df[col].value_counts(dropna=False).head(20))  # Топ-20 значений (+ пропуски)


--- base_category_name ---
base_category_name
Личные вещи                      135662
Для дома и дачи                  110919
Электроника                       77779
Готовый бизнес и оборудование     59149
Хобби и отдых                     52036
Транспорт                         51862
Животные                          12590
None                                  3
Name: count, dtype: int64

--- cand_category_name ---
cand_category_name
Личные вещи                      135662
Для дома и дачи                  110919
Электроника                       77782
Готовый бизнес и оборудование     59150
Хобби и отдых                     52035
Транспорт                         51862
Животные                          12590
Name: count, dtype: int64

--- base_subcategory_name ---
base_subcategory_name
Одежда, обувь, аксессуары      71424
Ремонт и строительство         62455
Оборудование для бизнеса       58547
Запчасти и аксессуары          51862
Детская одежда и обувь         34311
Мебель и интерье

In [ ]:
text_columns = ['base_title', 'cand_title', 'base_description', 'cand_description']

for col in text_columns:
    print(f"\n--- Примеры из {col} ---")
    print(df[col].head(5).values) 


--- Примеры из base_title ---
['Зимние ботинки ecco' 'Зимние ботинки ecco' 'Зимние ботинки ecco'
 'Куpткa зимняя и ветpовкa' 'Куpткa зимняя и ветpовкa']

--- Примеры из cand_title ---
['Кигуpуммии мышкa inextenso' 'Штaны для девочки zara'
 'Рубaшкa acoola 152' 'Зимние ботинки ecco' 'Плaщ детcкий next']

--- Примеры из base_description ---
['ботинки экко,униcекc,зимние\nноcилиcь один cезон,cтaли мaлы'
 'ботинки экко,униcекc,зимние\nноcилиcь один cезон,cтaли мaлы'
 'ботинки экко,униcекc,зимние\nноcилиcь один cезон,cтaли мaлы'
 'зимнюю куpтку ноcили меньше cезонa,почти новaя\nкуpткa зимa kerry-5300\n\nветpовкa didriksons-1200'
 'зимнюю куpтку ноcили меньше cезонa,почти новaя\nкуpткa зимa kerry-5300\n\nветpовкa didriksons-1200']

--- Примеры из cand_description ---
['.' 'штaны новые,ноcили пapу paз' 'новaя,не ноcили'
 'ботинки экко,униcекc,зимние\nноcилиcь один cезон,cтaли мaлы'
 '.пеpед отпpaвкой отпapю)']


In [ ]:
sample_json = df['base_json_params'].iloc[0]

try:
    parsed_json = json.loads(sample_json)
    print("\n--- Пример base_json_params ---")
    print(parsed_json)
except:
    print("Не удалось распарсить JSON. Возможно, данные в другом формате.")


--- Пример base_json_params ---
{'178': 758, '179': 768, '2756': 19916, '2827': 20032, '110064': [1659], '112674': 754147, '115582': 1509575, '115634': 1689223, '159501': 0, '166222': 3268045}


Числовые ключи (178, 179, 2756 и т. д.), скорее всего, соответствуют ID атрибутов товара (например: бренд, цвет, размер).

Можно проверить, сколько параметров совпадает у пар объявлений.

In [22]:
def compare_json_params(row):
    try:
        base_params = json.loads(row['base_json_params'])
        cand_params = json.loads(row['cand_json_params'])
        
        # Совпадающие ключи и значения
        common_keys = set(base_params.keys()) & set(cand_params.keys())
        same_values = sum(1 for k in common_keys if base_params[k] == cand_params[k])
        
        return {
            'common_params_count': len(common_keys),
            'same_values_count': same_values
        }
    except:
        return {'common_params_count': 0, 'same_values_count': 0}

comparison_results = df.apply(compare_json_params, axis=1, result_type='expand')
df = pd.concat([df, comparison_results], axis=1)

print("\nСреднее количество общих параметров:")
print(df['common_params_count'].mean())

print("\nСреднее количество совпадающих значений:")
print(df['same_values_count'].mean())


Среднее количество общих параметров:
7.408894

Среднее количество совпадающих значений:
5.775166


In [16]:
print("\n--- Пример base_title_image ---")
print(df['base_title_image'].iloc[0])

print("\n--- Пример cand_title_image ---")
print(df['cand_title_image'].iloc[0])


--- Пример base_title_image ---
40c72f08e0bb10b55e0605781481df2b5557b094aee695e471f7b6b2855a6cb2

--- Пример cand_title_image ---
ebc7537d69a1c8c1a6e7ea3c5b27ab4d4a360e6032d15877b33fb648c57dbf54


Похоже, что поля `base_title_image` и `cand_title_image` содержат хеш-суммы изображений (скорее всего, SHA-256). Это означает, что:
- Изображения уже предобработаны - хранятся не сами пиксели, а их хеши
- Сравнение упрощается - можно искать точные совпадения хешей
- Разные хеши ≠ разные изображения (но с высокой вероятностью)

In [26]:
# Процент пар с одинаковыми изображениями
same_images = (df['base_title_image'] == df['cand_title_image']).mean()
print(f"Точно одинаковые изображения: {same_images*100:.2f}%")

print(f"Уникальных base изображений: {df['base_title_image'].nunique()}")
print(f"Уникальных cand изображений: {df['cand_title_image'].nunique()}")

Точно одинаковые изображения: 1.34%
Уникальных base изображений: 191562
Уникальных cand изображений: 469367


In [18]:
print("\n--- Распределение is_same_location ---")
print(df['is_same_location'].value_counts())

print("\n--- Распределение is_same_region ---")
print(df['is_same_region'].value_counts())


--- Распределение is_same_location ---
is_same_location
True     489824
False     10176
Name: count, dtype: int64

--- Распределение is_same_region ---
is_same_region
True     492808
False      7192
Name: count, dtype: int64


In [21]:
print("Есть ли дубликаты в base_item_id?")
print(df['base_item_id'].duplicated().any())

print("Есть ли дубликаты в cand_item_id?")
print(df['cand_item_id'].duplicated().any())

Есть ли дубликаты в base_item_id?
True
Есть ли дубликаты в cand_item_id?
True
